In [1]:
import pandas as pd

df_train = pd.read_csv("train.csv")
# test = pd.read_csv("test.csv")

print(df_train.shape)
df_train.head()


(18153, 21)


,ID,ASI_category,Temperature,Precipitation,Rainfall,Snowfall,Soil_Temperature,Radiation,Wind_Speed,Wind_Gusts,...,Surface_Pressure,Relative_Humidity,Soil_Moisture,Dew_Point,Sunshine_Duration,Cloud_Cover,Precipitation_Hours,Wind_Direction,Weather_Code,Daylight_Duration
0,19554,Moderate,0.931231,0.000912,0.000912,0.0,0.757673,0.879671,0.179293,0.193029,...,0.538056,55,0.546243,17.564597,53252.08,12.136192,1,176.459082,51,58772.52
1,25205,Moderate,0.566323,0.096715,0.096715,0.0,0.291448,0.008913,0.588384,0.532172,...,0.568475,88,0.557803,5.692134,0.00,91.901341,16,232.433005,61,28143.12
2,771,Poor,0.018033,0.000000,0.000000,0.0,0.000000,0.277340,0.247475,0.189008,...,0.706520,78,0.791908,-25.264420,30213.79,18.859670,0,44.688600,3,34621.43
3,1976,Good,0.717541,0.000000,0.000000,0.0,0.635669,0.796709,0.123737,0.134048,...,0.547500,57,0.473988,5.913865,44627.21,38.759757,0,333.640418,3,59192.17
4,14036,Moderate,0.827170,0.001825,0.001825,0.0,0.743855,0.781282,0.343434,0.391421,...,0.546378,50,0.459538,9.661455,45267.17,60.058955,1,86.996954,51,59956.03


In [2]:
df_train.dtypes

ID                       int64
ASI_category            object
Temperature            float64
Precipitation          float64
Rainfall               float64
Snowfall               float64
Soil_Temperature       float64
Radiation              float64
Wind_Speed             float64
Wind_Gusts             float64
Pressure_MSL           float64
Surface_Pressure       float64
Relative_Humidity        int64
Soil_Moisture          float64
Dew_Point              float64
Sunshine_Duration      float64
Cloud_Cover            float64
Precipitation_Hours      int64
Wind_Direction         float64
Weather_Code             int64
Daylight_Duration      float64
dtype: object

In [3]:
df_train.isnull().sum()

ID                     0
ASI_category           0
Temperature            0
Precipitation          0
Rainfall               0
Snowfall               0
Soil_Temperature       0
Radiation              0
Wind_Speed             0
Wind_Gusts             0
Pressure_MSL           0
Surface_Pressure       0
Relative_Humidity      0
Soil_Moisture          0
Dew_Point              0
Sunshine_Duration      0
Cloud_Cover            0
Precipitation_Hours    0
Wind_Direction         0
Weather_Code           0
Daylight_Duration      0
dtype: int64

In [7]:
# %%
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode target labels
le = LabelEncoder()
y = le.fit_transform(df_train["ASI_category"])   # One-time encoding

# Split features and target
X = df_train.drop(["ID", "ASI_category"], axis=1)

# Split into train/validation sets (stratified for balance)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training set:", X_train.shape, "Validation set:", X_val.shape)
print("Classes:", le.classes_)


Training set: (14522, 19) Validation set: (3631, 19)
Classes: ['Good' 'Moderate' 'Poor']


In [13]:
import optuna
from sklearn.metrics import f1_score
from xgboost import XGBClassifier

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 600),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        # "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),   # fixed lower bound
        # "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True), # fixed lower bound
        "random_state": 42,
        "use_label_encoder": False,
        "eval_metric": "mlogloss",
    }

    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred, average='macro')
    return f1

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=True)

# Results
print(f"Best F1 Score: {study.best_value:.4f}")
print("Best Parameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")


[I 2025-10-30 01:35:51,568] A new study created in memory with name: no-name-ebb605ec-ddc9-4f47-83fa-d83359dabf23


  0%|          | 0/30 [00:00<?, ?it/s]

c:\Users\ommah\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:35:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-10-30 01:35:53,561] Trial 0 finished with value: 0.9191111711687996 and parameters: {'n_estimators': 446, 'learning_rate': 0.031188433299241893, 'max_depth': 3, 'subsample': 0.9148912716770974, 'colsample_bytree': 0.7494494001165428, 'gamma': 0.3052772668180853, 'min_child_weight': 6}. Best is trial 0 with value: 0.9191111711687996.


c:\Users\ommah\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:35:53] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


...

[I 2025-10-30 01:37:00,033] Trial 27 finished with value: 0.9198432221599403 and parameters: {'n_estimators': 548, 'learning_rate': 0.06941504203667716, 'max_depth': 7, 'subsample': 0.569149246478853, 'colsample_bytree': 0.7933410488550955, 'gamma': 0.7079350910854867, 'min_child_weight': 8}. Best is trial 11 with value: 0.9224810429509563.


c:\Users\ommah\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:37:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-10-30 01:37:01,926] Trial 28 finished with value: 0.9183353489509228 and parameters: {'n_estimators': 352, 'learning_rate': 0.17578465395530327, 'max_depth': 8, 'subsample': 0.6159355292074354, 'colsample_bytree': 0.8314676531054369, 'gamma': 2.389179159305936, 'min_child_weight': 5}. Best is trial 11 with value: 0.9224810429509563.


c:\Users\ommah\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:37:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-10-30 01:37:06,535] Trial 29 finished with value: 0.9159897518059336 and parameters: {'n_estimators': 441, 'learning_rate': 0.028471337138666078, 'max_depth': 10, 'subsample': 0.8792459174915441, 'colsample_bytree': 0.668458979658134, 'gamma': 0.3865145045861845, 'min_child_weight': 6}. Best is trial 11 with value: 0.9224810429509563.
Best F1 Score: 0.9225
Best Parameters:
  n_estimators: 456
  learning_rate: 0.06516782670312797
  max_depth: 9
  subsample: 0.5016525286135156
  colsample_bytree: 0.8767039096809168
  gamma: 1.1549726203545656
  min_child_weight: 4


In [14]:
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

def objective_lgbm(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 600),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 15, 80),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),  # L1
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True), # L2
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
        "random_state": 42,
        "n_jobs": -1,
    }

    model = LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return f1_score(y_val, y_pred, average="macro")

# Run Optuna study
study_lgbm = optuna.create_study(direction="maximize")
study_lgbm.optimize(objective_lgbm, n_trials=50, show_progress_bar=True)

print("Best LGBM F1:", study_lgbm.best_value)
print("Best LGBM Params:", study_lgbm.best_params)


[I 2025-10-30 01:37:11,812] A new study created in memory with name: no-name-bdb58da1-1a9a-4dd3-bdfa-53f6cbeab2fe


  0%|          | 0/50 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002146 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3847
[LightGBM] [Info] Number of data points in the train set: 14522, number of used features: 19
[LightGBM] [Info] Start training from score -1.755382
[LightGBM] [Info] Start training from score -0.355142
[LightGBM] [Info] Start training from score -2.070802
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import optuna

def objective_rf(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 800),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 6),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 3),
        # "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        # "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        # "class_weight": trial.suggest_categorical("class_weight", [None, "balanced"]),
        "random_state": 42,
        "n_jobs": -1,
    }

    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    return f1_score(y_val, y_pred, average="macro")

# Run Optuna study
study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_rf, n_trials=50, show_progress_bar=True)

print("\n✅ Best Random Forest F1:", study_rf.best_value)
print("Best Random Forest Params:")
print(study_rf.best_params)


[I 2025-10-30 21:13:51,105] A new study created in memory with name: no-name-b9675851-ac74-4865-8864-cb72557a31be


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-30 21:13:53,799] Trial 0 finished with value: 0.747357330060875 and parameters: {'n_estimators': 474, 'max_depth': 4, 'min_samples_split': 5, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.747357330060875.
[I 2025-10-30 21:13:58,448] Trial 1 finished with value: 0.7950559259669842 and parameters: {'n_estimators': 688, 'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.7950559259669842.
[I 2025-10-30 21:14:02,642] Trial 2 finished with value: 0.745472037610316 and parameters: {'n_estimators': 736, 'max_depth': 4, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.7950559259669842.
[I 2025-10-30 21:14:07,395] Trial 3 finished with value: 0.883108664138883 and parameters: {'n_estimators': 529, 'max_depth': 8, 'min_samples_split': 5, 'min_samples_leaf': 3}. Best is trial 3 with value: 0.883108664138883.
[I 2025-10-30 21:14:10,915] Trial 4 finished with value: 0.7471798684366333 and parameters: {'n_estim

In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import numpy as np

# ✅ Best parameters from Optuna
best_rf_params = {
    "n_estimators": 450,
    "max_depth": 9,
    "min_samples_split": 5,
    "min_samples_leaf": 2,
    "random_state": 42,
    "n_jobs": -1,
    "criterion":"log_loss",
    "max_features":0.7,
    "max_leaf_nodes":None,
    
}

# --- Train model ---
rf_best = RandomForestClassifier(**best_rf_params)
rf_best.fit(X_train, y_train)

# --- Predictions ---
train_preds = rf_best.predict(X_train)
val_preds = rf_best.predict(X_val)

# --- Metrics ---
train_acc = accuracy_score(y_train, train_preds)
val_acc = accuracy_score(y_val, val_preds)
train_f1 = f1_score(y_train, train_preds, average="macro")
val_f1 = f1_score(y_val, val_preds, average="macro")

print("\n✅ Random Forest Model Performance:")
print(f"Training Accuracy : {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Training F1 Score  : {train_f1:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")
print(f"Δ F1 Gap           : {abs(train_f1 - val_f1):.4f}")




✅ Random Forest Model Performance:
Training Accuracy : 0.9638
Validation Accuracy: 0.9369
Training F1 Score  : 0.9514
Validation F1 Score: 0.9138
Δ F1 Gap           : 0.0377


In [8]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

# --- XGBoost best params ---
best_xgb = XGBClassifier(
    n_estimators=528,
    learning_rate=0.06533478035634971, 
    max_depth=7, 
    subsample=0.5921834617441386, 
    colsample_bytree=0.6340862288557617, 
    gamma=1.399534210191034, 
    min_child_weight=3,
    random_state=42,
    use_label_encoder=False,
    eval_metric="mlogloss",
    n_jobs=-1
)

# --- LightGBM best params ---
best_lgbm = LGBMClassifier(
    n_estimators=255, 
    learning_rate=0.07817378543966169, 
    num_leaves=34, 
    max_depth=12, 
    subsample=0.7944500973942142, 
    colsample_bytree=0.9574183959836607, 
    reg_alpha=1.2227075268899349e-05, 
    reg_lambda=0.020009711326624165, 
    min_child_samples=18,
    random_state=42,
    n_jobs=-1
)

# --- Random Forest best params ---
best_rf = RandomForestClassifier(
    n_estimators=450,
    max_depth=9,
    min_samples_split=5,
    min_samples_leaf=2,
    criterion="log_loss",
    max_features=0.7,
    random_state=42,
    n_jobs=-1
)

# --- Voting Ensemble (Soft Voting) ---
voting_clf = VotingClassifier(
    estimators=[
        ("xgb", best_xgb),
        ("lgbm", best_lgbm),
        ("rf", best_rf)
    ],
    voting="soft",
    weights=[0.001, 0.95, 0.46],   # tweakable weights based on performance
    n_jobs=-1
)

# --- Train ---
voting_clf.fit(X_train, y_train)

# --- Predict ---
train_preds = voting_clf.predict(X_train)
val_preds = voting_clf.predict(X_val)

# --- Evaluate ---
train_acc = accuracy_score(y_train, train_preds)
val_acc = accuracy_score(y_val, val_preds)
train_f1 = f1_score(y_train, train_preds, average="macro")
val_f1 = f1_score(y_val, val_preds, average="macro")

print("\n✅ Voting Ensemble (XGB + LGBM + RF) Performance:")
print(f"Training Accuracy : {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Training F1 Score  : {train_f1:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")
print(f"Δ F1 Gap           : {abs(train_f1 - val_f1):.4f}")

# Optional: classification report
print("\nClassification Report:\n", classification_report(y_val, val_preds))



✅ Voting Ensemble (XGB + LGBM + RF) Performance:
Training Accuracy : 1.0000
Validation Accuracy: 0.9441
Training F1 Score  : 1.0000
Validation F1 Score: 0.9244
Δ F1 Gap           : 0.0756

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.89      0.90       628
           1       0.96      0.96      0.96      2546
           2       0.93      0.90      0.91       457

    accuracy                           0.94      3631
   macro avg       0.93      0.92      0.92      3631
weighted avg       0.94      0.94      0.94      3631




Voting Ensemble F1 (Fixed Weights): 0.9183689728576178


NameError: name 'train_preds' is not defined

In [48]:
from sklearn.metrics import f1_score, accuracy_score, mean_squared_error
import numpy as np

# Validate on hold-out set
y_pred = voting_clf.predict(X_val)

# Compute metrics
f1 = f1_score(y_val, y_pred, average="macro")
acc = accuracy_score(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

# Print results
print("📊 Validation Metrics:")
print(f"F1 Score (macro): {f1:.4f}")
print(f"Accuracy: {acc:.4f}")
print(f"RMSE: {rmse:.4f}")


📊 Validation Metrics:
F1 Score (macro): 0.9252
Accuracy: 0.9449
RMSE: 0.2364


In [49]:
# Retrain the final ensemble on 100% of the available training data
X_full = df_train.drop(["ID", "ASI_category"], axis=1)
y_full = le.transform(df_train["ASI_category"])  # use same label encoder

voting_clf.fit(X_full, y_full)
print("✅ Final ensemble retrained on all training data.")


c:\Users\ommah\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:03:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002056 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3871
[LightGBM] [Info] Number of data points in the train set: 18153, number of used features: 19
[LightGBM] [Info] Start training from score -1.755250
[LightGBM] [Info] Start training from score -0.355110
[LightGBM] [Info] Start training from score -2.071158
✅ Final ensemble retrained on all training data.


In [51]:
test = pd.read_csv("test.csv")
X_test = test.drop(["ID"], axis=1)

final_pred = voting_clf.predict(X_test)
final_pred = le.inverse_transform(final_pred)

submission = pd.DataFrame({
    "ID": test["ID"],
    "Target": final_pred
})

submission.to_csv("submission.csv", index=False)
print("✅ Submission file created successfully: submission.csv")
print(submission.head())


✅ Submission file created successfully: submission.csv
      ID    Target
0  15628      Good
1   9358  Moderate
2  12927  Moderate
3  23980      Poor
4   1032  Moderate


In [9]:
# %% -----------------------------------
# 🚀 Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, mean_squared_error, classification_report
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# %% -----------------------------------
# 📦 Load and preprocess data
df_train = pd.read_csv("train.csv")

# Encode target
le = LabelEncoder()
y = le.fit_transform(df_train["ASI_category"])
X = df_train.drop(["ID", "ASI_category"], axis=1)

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.05, stratify=y, random_state=42
)

# %% -----------------------------------
# ⚙️ Define Tuned Models

# --- XGBoost ---
best_xgb = XGBClassifier(
    n_estimators=528,
    learning_rate=0.06533478035634971, 
    max_depth=7, 
    subsample=0.5921834617441386, 
    colsample_bytree=0.6340862288557617, 
    gamma=1.399534210191034, 
    min_child_weight=3,
    random_state=42,
    use_label_encoder=False,
    eval_metric="mlogloss",
    n_jobs=-1
)

# --- LightGBM ---
best_lgbm = LGBMClassifier(
    n_estimators=255, 
    learning_rate=0.07817378543966169, 
    num_leaves=34, 
    max_depth=12, 
    subsample=0.7944500973942142, 
    colsample_bytree=0.9574183959836607, 
    reg_alpha=1.2227075268899349e-05, 
    reg_lambda=0.020009711326624165, 
    min_child_samples=18,
    random_state=42,
    n_jobs=-1
)

# --- Random Forest ---
best_rf = RandomForestClassifier(
    n_estimators=450,
    max_depth=9,
    min_samples_split=5,
    min_samples_leaf=2,
    criterion="log_loss",
    max_features=0.7,
    random_state=42,
    n_jobs=-1
)

# %% -----------------------------------
# 🧠 Voting Ensemble
voting_clf = VotingClassifier(
    estimators=[
        ("xgb", best_xgb),
        ("lgbm", best_lgbm),
        ("rf", best_rf)
    ],
    voting="soft",
    weights=[0.1, 0.7, 0.2],  # ← You can tune these manually if needed
    n_jobs=-1
)

# Train
voting_clf.fit(X_train, y_train)

# %% -----------------------------------
# 📊 Evaluate on Validation Set
y_pred = voting_clf.predict(X_val)

f1 = f1_score(y_val, y_pred, average="macro")
acc = accuracy_score(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print("\n📊 Validation Metrics:")
print(f"F1 Score (macro): {f1:.4f}")
print(f"Accuracy: {acc:.4f}")
print(f"RMSE: {rmse:.4f}")
print("\nClassification Report:\n", classification_report(y_val, y_pred))

# %% -----------------------------------
# 🔁 Retrain Final Ensemble on Full Data
X_full = df_train.drop(["ID", "ASI_category"], axis=1)
y_full = le.transform(df_train["ASI_category"])

voting_clf.fit(X_full, y_full)
print("\n✅ Final ensemble retrained on all training data.")

# %% -----------------------------------
# 🧾 Generate Submission
test = pd.read_csv("test.csv")
X_test = test.drop(["ID"], axis=1)

final_pred = voting_clf.predict(X_test)
final_pred = le.inverse_transform(final_pred)

submission = pd.DataFrame({
    "ID": test["ID"],
    "Target": final_pred
})

submission.to_csv("submission.csv", index=False)
print("\n✅ Submission file created successfully: submission.csv")
print(submission.head())



📊 Validation Metrics:
F1 Score (macro): 0.9179
Accuracy: 0.9405
RMSE: 0.2506

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.90      0.90       157
           1       0.96      0.96      0.96       637
           2       0.90      0.89      0.89       114

    accuracy                           0.94       908
   macro avg       0.92      0.92      0.92       908
weighted avg       0.94      0.94      0.94       908


✅ Final ensemble retrained on all training data.

✅ Submission file created successfully: submission.csv
      ID    Target
0  15628      Good
1   9358  Moderate
2  12927  Moderate
3  23980      Poor
4   1032  Moderate
